Perform CIFAR10 classification using ResNet18.

1) Train ResNet18 from Scratch


2) Finetune ResNet from Pretarined network on ImageNet


In [9]:
!pip install torchmetrics


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models,transforms
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import torchmetrics as tm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 30.1 MB/s 


In [10]:
# CIFAR10
def cifar10(batch_sz, path='./datasets'):
    num_classes = 10
    transform_train = transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                    ])
    transform_test = transforms.Compose([
                        transforms.ToTensor(),
                    ])

    # Training dataset
    train_data = CIFAR10(root=path, train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_sz,
                                               shuffle=True, pin_memory=True)

    # Test dataset
    test_data = CIFAR10(root=path, train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_sz, shuffle=False, pin_memory=True)

    return train_loader, test_loader, num_classes


In [11]:
batch_sz=64 # this is batch size i.e. the number of rows in a batch of data
train_loader, test_loader, _=cifar10(batch_sz) 

Files already downloaded and verified
Files already downloaded and verified


In [12]:
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

print(device)

cuda:0


In [14]:
model.fc == nn.Linear(in_features=512, out_features=10, bias=True)

False

In [21]:
# Hyperparameter

learning_rate = 1e-3
mm = 0
batch_sz=512
epoch_no = 10



In [ ]:
model = model.to(device=device)

# write to tensorboard
step = 0
writer = SummaryWriter(f'runs/ResNet/bs={batch_sz}_lr={learning_rate}')


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum= mm)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer,gamma=0.9,verbose=False)

for epoch in range(epoch_no):
    train_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forwards
        logits = model(data)
        loss = criterion(logits, targets)
        train_loss += loss.item()


        # backward 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        # Running Accuracy
        _, predictions = logits.max(1)
        num_corr = (predictions == targets).sum()
        running_acc = float(num_corr)/float(data.shape[0])
        
        writer.add_scalar("Training Loss", loss, global_step = step) 
        writer.add_scalar("Training Accuracy", running_acc, global_step=step)
        step += 1  

    scheduler.step()

acc_l = []

accuracy = tm.Accuracy()

with torch.no_grad():
    step_2 = 0
    for batch_idx, (data, targets) in enumerate(test_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        logits = model(data)
        t_loss = criterion(logits, targets)
        train_loss += loss.item()

        yhat = torch.argmax(logits, axis =1)

        acc = accuracy(yhat.to("cpu"),targets.to("cpu"))

        acc_l.append(acc)

        writer.add_scalar("Testing Loss", t_loss, global_step = step_2) 
        writer.add_scalar("Testing Accuracy", acc, global_step=step_2)
        step_2 += 1

print(f'the accuracy on the test set for the batch size: {batch_sz} and learning rate: {learning_rate} is {np.mean(acc_l):.2f}')